# Resnet Finetuning on MSCOCO

- add 2 optimizers, one for mscoco, one for top layer

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pickle
import os
import shutil
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.contrib import slim
from utils.data import Dataset
from utils.data import init_dir

## Load data

In [ ]:
class data:
    train = Dataset('/data/mscoco/train_data.hdf', '/data/mscoco/train_imgs/', one_hot=True)
    validation = Dataset('/data/mscoco/valid_data.hdf', '/data/mscoco/valid_imgs/', one_hot=True)
    
class_id2class_name_mapping = {
    0: 'cow',
    1: 'sheep',
    2: 'giraffe',
    3: 'horse',
    4: 'bird',
    5: 'cat',
    6: 'dog',
    7: 'elephant',
    8: 'bear'}

## Build Resnet Graph

In [ ]:
from utils.nn_graph import simple_layer, variable_summaries

In [ ]:
resnet_model_ckpt = '/data/checkpoints/resnet_v1_50.ckpt'
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('resnet_inputs'):
        images = tf.placeholder(dtype=tf.float32, shape=[None, 224 * 224 * 3], name='images')
        labels = tf.placeholder(tf.float32, shape=(None, 9), name='labels')
        is_training = tf.placeholder(tf.bool, shape=(), name='training_flag')

    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 224, 224, 3])        

    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        resnet_tensors = resnet_v1.resnet_v1_50(images_reshaped,
                                                is_training=is_training,
                                                num_classes=None,
                                                global_pool=True,
                                                scope='resnet_v1_50')
    resnet_feature_vector, resnet_endpoints = resnet_tensors

    with tf.name_scope('resnet_feature_vector'):
        resnet_feature_vector = tf.reshape(resnet_feature_vector, shape=(-1, 2048))

    with tf.variable_scope('layer1'):
        prediction_raw = resnet_feature_vector
        prediction_raw = simple_layer('layer1', prediction_raw, shape=[2048, 9], activation='linear')
        
    ##################
    # Add relu layer #
    ##################

    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(prediction_raw)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.losses.softmax_cross_entropy(logits=prediction_raw, onehot_labels=labels)
        variable_summaries('loss_summary', cross_entropy_vector)
        loss = tf.reduce_mean(cross_entropy_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)
        
    with tf.name_scope("embedding_visualization"):
        embedding = tf.Variable(tf.zeros([480, 2048]), name='valid_embedding')
        embedding_assignment = embedding.assign(resnet_feature_vector)       
    
    with tf.name_scope('training'):
        loss_to_optimize = loss
        
        var_all = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        var_resnet = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='resnet_v1_50')
        
        ######################################################################
        # Prepare 2 optimizers, one with lr=0.0001 for resnet block 3 and 4, #
        # one for layer1 and layer2 with lr=0.001                            #
        ######################################################################
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            #train_step = tf.group(train_resnet, train_top)
            train_step = tf.train.AdamOptimizer(0.0001).minimize(loss_to_optimize)
            
    init_resnet = slim.assign_from_checkpoint_fn(resnet_model_ckpt, slim.get_model_variables('resnet_v1_50'))
    initialize_vars = tf.group(
        tf.global_variables_initializer(),
        tf.local_variables_initializer())
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/resnet/'
exp_name = 'experiment_2'

logging_meta = init_model_logging(base_dir, exp_name, graph=graph, remove_existing=True)

## Add Embedding Projection

In [ ]:
from utils.nn_visualization import init_embedding_projector, init_embedding_data
from utils.nn_visualization import get_sprite_img, get_label_class_names
projection_img_shape = [80, 80, 3]

In [ ]:
data.validation.image_shape = projection_img_shape
sprite_img = get_sprite_img(data.validation.images, projection_img_shape)
data.validation.image_shape = [224, 224, 3]

label_names = get_label_class_names(data.validation.labels, class_id2class_name_mapping)

In [ ]:
init_embedding_data(logging_meta['valid_writer_dir'], sprite_img, label_names)
init_embedding_projector(logging_meta['valid_writer'], embedding, projection_img_shape)

## Run Resnet

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']


validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    is_training: False}

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    init_resnet(session)
    for iteration in range(5):
        _images, _labels = data.train.next_batch(30)
        
        feed_dict={images: _images, 
                   labels: _labels,
                   is_training: True}
        _ = session.run([train_step], feed_dict)
        
        if iteration % 1 == 0:
            feed_dict={images: _images, 
                       labels: _labels,
                       is_training: False}
            _summary,  _accuracy, _loss = session.run([merge_summaries, accuracy, loss], feed_dict)
            print("Iteration Train {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
            logging_meta['train_writer'].add_summary(_summary, iteration)
            
        if iteration > 0 and iteration % 4 == 0:
            fetches = [embedding_assignment, merge_summaries, accuracy, loss]
            _, _summary, _accuracy, _loss = session.run(fetches, validation_feed_dict)
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
    
    _prediction, = session.run([prediction], validation_feed_dict)

## Results evaluation

In [ ]:
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)